In [1]:
import torch

In [6]:

# Define logical statements
A = torch.tensor([1.0], requires_grad=True)
B = torch.tensor([0.0], requires_grad=True)

# Differentiable AND operation using a loss function
input_tensor = torch.stack([A, B])
target_tensor = torch.tensor([1.0, 0.0]).reshape(input_tensor.shape)  # Reshape target tensor to match input tensor size

loss = torch.nn.functional.binary_cross_entropy_with_logits(input_tensor, target_tensor)

# Propagate through the graph
loss.backward()

# Print gradients
print("Gradient of A:", A.grad.item())
print("Gradient of B:", B.grad.item())


Gradient of A: -0.13447070121765137
Gradient of B: 0.25


In [8]:
import random
from pgmpy.models import BayesianModel
from pgmpy.estimators import ParameterEstimator
from pgmpy.sampling import BayesianModelSampling

# Set the number of nodes and maximum number of parents
nb_nodes = 38
nb_parent_max = 3

# Initialize the Bayesian Model
dag = BayesianModel()

# Generate the random graph
for node in range(nb_nodes, 1, -1):
    # Ensure that the sample size is within bounds
    sample_size = min(random.randint(0, nb_parent_max), node - 1)
    
    N_parent = random.sample(range(node), sample_size)
    dag.add_nodes_from([node])
    dag.add_edges_from([(parent, node) for parent in N_parent])

# To avoid isolated nodes
for i in range(1, nb_nodes + 1):
    ps = dag.predecessors(i)
    cs = dag.successors(i)
    if not ps and not cs:
        if i == 1:
            N_children = random.sample(range(1, nb_nodes + 1), min(2, nb_nodes))
            dag.add_edges_from([(i, N_children[0]), (i, N_children[1])])
        else:
            N_parent = random.sample(range(1, i), min(1, i - 1))
            dag.add_edges_from([(N_parent[0], i)])

# Convert the directed acyclic graph to pgmpy format
model = BayesianModel(dag.edges())

# Randomly assign some CPDs
for node in model.nodes():
    parents = list(model.predecessors(node))
    cardinality = 2  # Binary variables
    cpd_values = [random.expovariate(1) for _ in range(cardinality**len(parents))]
    model.add_cpds(node, parents, cpd_values)

# Generate random evidence
evidence = {f"Variable_{random.randint(1, nb_nodes)}": random.randint(0, 1) for _ in range(5)}

# Perform Bayesian inference
inference = BayesianModelSampling(model)
samples = inference.forward_sample(size=1000)

# Estimate parameters
parameter_estimator = ParameterEstimator(model, samples)
cpds = parameter_estimator.get_parameters()

# Print CPDs
for node, cpd in cpds.items():
    print(f"Node {node} CPD:")
    print(cpd)

# Perform further analysis or computations as needed


ValueError: Only TabularCPD or ContinuousFactor can be added.